In [1]:
# import the data
import pandas as pd

training = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

training['train_test'] = 1
test['train_test'] = 0
all_data = pd.concat([training,test])

print("Import Data Complete")

Import Data Complete


In [2]:
training.head(10)
training.tail(10)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,train_test
57467,4294254797,gpt-4-1106-preview,claude-2.0,"[""can you provide a modular python code that w...","[""Certainly! Below is a modular Python example...","[""Here is a modular Python code to format SQL ...",1,0,0,1
57468,4294349169,palm-2,guanaco-33b,"[""Schreibe ein Bewerbungsanschreiben in deutsc...","[""I am trained to understand and respond only ...","[""Here is a possible Bewerbungsanschreiben in ...",0,1,0,1
57469,4294452769,gpt-4-1106-preview,openchat-3.5,"[""Old McDonald harvested 4 pumpkins this year....","[""Old McDonald originally harvested 4 pumpkins...","[""Old McDonald had 4 pumpkins initially and no...",0,0,1,1
57470,4294602932,claude-2.1,gpt-4-1106-preview,"[""I want you to act like {character} from {ser...","[""*responds in character's voice* Well hello t...","[""Of course! However, you'll need to specify w...",0,1,0,1
57471,4294633312,pplx-7b-online,gpt-4-0613,"[""abc""]","[""It seems like you're referring to the Califo...","[""Hello! It seems like you've typed \""abc.\"" H...",1,0,0,1
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0,1
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0,1
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0,1
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0,1
57476,4294947231,gemini-pro-dev-api,gpt-4-1106-preview,"[""three kids eat three apples in three days, h...","[""27 apples""]","[""If three kids eat three apples in three days...",1,0,0,1


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Separate vectorizers for each field
vectorizer_prompt = TfidfVectorizer(max_features=150)
vectorizer_response_a = TfidfVectorizer(max_features=150)
vectorizer_response_b = TfidfVectorizer(max_features=150)

# Fit on training data
temp_prompt = vectorizer_prompt.fit_transform(training["prompt"])
temp_response_a = vectorizer_response_a.fit_transform(training["response_a"])
temp_response_b = vectorizer_response_b.fit_transform(training["response_b"])

print("TF-IDF Features for Prompt:", vectorizer_prompt.get_feature_names_out())
print("TF-IDF Features for Response A:", vectorizer_response_a.get_feature_names_out())
print("TF-IDF Features for Response B:", vectorizer_response_b.get_feature_names_out())

print("Number of elements for the vectorizer representation for 'prompt':", temp_prompt.shape)
print("Number of elements for the vectorizer representation for 'response a':", temp_response_a.shape)
print("Number of elements for the vectorizer representation for 'response b':", temp_response_b.shape)

#selecting the prediction target
def get_winner(row):
    if row['winner_model_a'] == 1:
        return 0  # response_a wins
    elif row['winner_model_b'] == 1:
        return 1  # response_b wins
    elif row['winner_tie'] == 1:
        return 2  # tie
    else:
        return -1  # unknown or missing

training['winner'] = training.apply(get_winner, axis=1)
train_y = training["winner"].values

#choosing "features"
train_X = np.concatenate((temp_prompt.toarray(), temp_response_a.toarray(), temp_response_b.toarray()), axis=1)

print("Selecting The Prediction Target and Choosing Features Complete")
#use Logistic Regression
from sklearn.linear_model import LogisticRegression
from datetime import datetime

#record start time to calculate the execution time
start = datetime.now()

#Logistic Regression
model = LogisticRegression(max_iter=500, multi_class='multinomial', solver='saga') #For large datasets the “saga” solver is usually faster [scikit-learn documentation]
model.fit(train_X, train_y)

#record end time
end = datetime.now()
 
#calculate the execution time
execution_time = (end - start).total_seconds() / 60
print(f"The time of execution is: {execution_time} minutes")


print("Model Training Complete")

TF-IDF Features for Prompt: ['10' 'about' 'after' 'ai' 'all' 'also' 'am' 'an' 'and' 'answer' 'any'
 'are' 'as' 'at' 'based' 'be' 'been' 'being' 'best' 'between' 'but' 'by'
 'can' 'code' 'comment' 'could' 'create' 'data' 'day' 'do' 'does' 'don'
 'each' 'example' 'explain' 'first' 'following' 'for' 'from' 'game' 'get'
 'give' 'good' 'had' 'has' 'have' 'he' 'help' 'her' 'his' 'how' 'if' 'in'
 'information' 'int' 'into' 'is' 'it' 'its' 'just' 'know' 'like' 'list'
 'long' 'make' 'many' 'me' 'model' 'more' 'most' 'must' 'my' 'name' 'need'
 'new' 'no' 'not' 'now' 'nthe' 'number' 'of' 'on' 'one' 'only' 'or'
 'other' 'our' 'out' 'people' 'please' 'provide' 'python' 'question'
 'return' 'self' 'she' 'should' 'so' 'some' 'step' 'story' 'such' 'system'
 't0' 'take' 'tell' 'text' 'than' 'that' 'the' 'their' 'them' 'then'
 'there' 'these' 'they' 'think' 'this' 'time' 'to' 'two' 'u2019s' 'u201d'
 'up' 'use' 'user' 'using' 'very' 'want' 'was' 'way' 'we' 'were' 'what'
 'when' 'where' 'which' 'who' 'why

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

#split into validation and training data
train_X_train, train_X_val, train_y_train, train_y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

#record start time to calculate the execution time
start = datetime.now()

#think about results - comparing predictions (value_y_predict) to the actual winner model (train_y_val)
value_y_predict = model.predict(train_X_val)
print('Model winner prediction', value_y_predict)
print('Model winner real value', train_y_val)

value_y_probabilities = model.predict_proba(train_X_val)
print('Model winner prediction, probability', value_y_probabilities) #winner model a | winner model b | winner tie

#confusion matrix
cm = confusion_matrix(train_y_val, value_y_predict)
print("Confusion Matrix:\n", cm)

#model accuracy
score = model.score(train_X_val, train_y_val)
print('Model Accuracy Score', score)

#macro and micro averaged Precision and Recall
macro_precision = precision_score(train_y_val, value_y_predict, average='macro') #calculate precision for all classes individually and then average them
macro_recall = recall_score(train_y_val, value_y_predict, average='macro')
micro_precision = precision_score(train_y_val, value_y_predict, average='micro') #calculate class wise true positive and false positive and then use that to calculate overall precision
micro_recall = recall_score(train_y_val, value_y_predict, average='micro')
print("Macro Precision:", macro_precision)
print("Macro Recall:", macro_recall)
print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

#record end time
end = datetime.now()
 
#calculate the execution time
execution_time = (end - start).total_seconds()
print(f"The time of execution is: {execution_time} seconds")

Model winner prediction [0 1 2 ... 1 0 0]
Model winner real value [0 0 2 ... 1 0 0]
Model winner prediction, probability [[0.41029326 0.34035515 0.24935159]
 [0.21808727 0.54393733 0.2379754 ]
 [0.28465304 0.26706572 0.44828124]
 ...
 [0.26382212 0.49364626 0.24253162]
 [0.40534063 0.30900811 0.28565126]
 [0.54751897 0.17925463 0.2732264 ]]
Confusion Matrix:
 [[2128 1186  716]
 [1267 1990  672]
 [1228 1152 1157]]
Model Accuracy Score 0.45885525400139177
Macro Precision: 0.45824024285175624
Macro Recall: 0.4538810920723748
Micro Precision: 0.45885525400139177
Micro Recall: 0.45885525400139177
The time of execution is: 0.126692 seconds


In [5]:
#model log loss - https://www.kaggle.com/competitions/llm-classification-finetuning/discussion/552103
from sklearn.metrics import log_loss

model_log_loss = log_loss(train_y_val, value_y_probabilities)

print('Model Log loss:', model_log_loss) 

# Number of classes = 3 : Logloss = - log(1/3) = 1.10
# Model Log loss: 1.05, model prediction is considered good for this project

Model Log loss: 1.0465078397010954


In [6]:
#model prediction
# transform test data using the trained vectorizers
temp_test_prompt = vectorizer_prompt.transform(test["prompt"])
temp_test_response_a = vectorizer_response_a.transform(test["response_a"])
temp_test_response_b = vectorizer_response_b.transform(test["response_b"])

test_X = np.concatenate((temp_test_prompt.toarray(), temp_test_response_a.toarray(), temp_test_response_b.toarray()), axis=1)
value_test_y_probabilities = model.predict_proba(test_X)
print('Model winner prediction, probability', value_test_y_probabilities) #winner model a | winner model b | winner tie

Model winner prediction, probability [[0.21811121 0.20748486 0.57440394]
 [0.45421621 0.26237458 0.2834092 ]
 [0.26653411 0.38061243 0.35285347]]


In [7]:
output = pd.DataFrame({'id': test.id,
                        'winner_model_a': value_test_y_probabilities[:, 0],
                        'winner_model_b': value_test_y_probabilities[:, 1],
                        'winner_tie': value_test_y_probabilities[:, 2]})
output.to_csv('submission.csv', index=False)